## Project 2 - Casimir Funk



In [1]:
#!pip install -r requirements.txt
import pandas as pd
import numpy as np

### 1.1 #A Description of population of interest.

Population of San Francisco and comparing the difference in minimum cost diets of people across different diet groups, such as vegetarian, pescatarian, vegan, meat eater, etc.

### 1.2 #A Dietary Reference Intakes

Write a function that takes as arguments the characterists of a person (e.g. age, sex) and returns a pandas.Series of Dietary Reference Intakes (DRI's) or "Recommended Daily Allowances" (RDA) of a variety of nutrients appropriate for your population of interest.

In [2]:
def DRI(age=19, sex='M'):
    bmin = pd.read_csv('./diet_minimums.csv').set_index('Nutrition').iloc[:,2:]
    bmax = pd.read_csv('./diet_maximums.csv').set_index('Nutrition').iloc[:,2:]
    combined_df = pd.concat([bmin,-bmax])
    if sex == "M":
        if age <=3:
            return combined_df.loc[:, 'C 1-3']
        if age>=4 and age<=8:
            return combined_df.loc[:, 'M 4-8']
        if age>=9 and age<=13:
            return combined_df.loc[:, 'M 9-13']
        if age>= 14 and age<=18:
            return combined_df.loc[:,'M 14-18']
        if age>=19 and age <= 30: 
            return combined_df.loc[:, 'M 19-30']
        if age>=31 and age<=50:
            return combined_df.loc[:, 'M 31-50']
        if age>=51:
            return combined_df.loc[:, 'M 51+']
    if sex == "F":
        if age <=3:
            return combined_df.loc[:, 'C 1-3']
        if age>=4 and age<=8:
            return combined_df.loc[:, 'F 4-8']
        if age>=9 and age<=13:
            return combined_df.loc[:, 'F 9-13']
        if age>= 14 and age<=18:
            return combined_df.loc[:,'F 14-18']
        if age>=19 and age <= 30: 
            return combined_df.loc[:, 'F 19-30']
        if age>=31 and age<=50:
            return combined_df.loc[:, 'F 31-50']
        if age>=51:
            return combined_df.loc[:, 'F 51+']
DRI(25, 'M')

Nutrition
Energy                            2400.0
Protein                             56.0
Fiber, total dietary                33.6
Folate, DFE                        400.0
Calcium, Ca                       1000.0
Carbohydrate, by difference        130.0
Iron, Fe                             8.0
Magnesium, Mg                      400.0
Niacin                              16.0
Phosphorus, P                      700.0
Potassium, K                      4700.0
Riboflavin                           1.3
Thiamin                              1.2
Vitamin A, RAE                     900.0
Vitamin B-12                         2.4
Vitamin B-6                          1.3
Vitamin C, total ascorbic acid      90.0
Vitamin E (alpha-tocopherol)        15.0
Vitamin K (phylloquinone)          120.0
Zinc, Zn                            11.0
Sodium, Na                       -2300.0
Name: M 19-30, dtype: float64

### 1.3 #A Data on prices for different foods (Google Sheets)

API user and keys:

In [3]:
user = "project2-casimir-funk"

# API key for Gov
apikey = "V0mOAdVrSineT4d2VTMTJUNPAsBTnjzUAU6e6H6V" # inIyO1begWSRqsYtxS7m6p09PSyq7Qiw7fxzV2qN"

# File with private keys for relevant service account to authenticate
# and access google spreadsheets
!gpg --passphrase "casimir" -d project2-casimir-funk-306520-ae3703c35882.json.gpg > project2-casimir-funk-306520-ae3703c35882.json

serviceacct = {'project2-casimir-funk':'project2-casimir-funk-306520-ae3703c35882.json'}
sheet_url = "https://docs.google.com/spreadsheets/d/1XqnsamAhXq2xnktB_rJ8h8ovvpFQdWWcDkx_1Yrfqdg/edit#gid=0"


gpg: AES256 encrypted data
gpg: encrypted with 1 passphrase


#### Reading data and diet types from google sheet

In [4]:
# sheet: diet type
diet_type = 'mcdonalds'

In [5]:
from eep153_tools import read_sheets

# Accessing google sheets and convert to pandas.dataframe
data = read_sheets(key=sheet_url, json_creds=serviceacct[user], sheet=diet_type)
headers = data.iloc[0]
df  = pd.DataFrame(data.values[1:], columns=headers)
df

,Category,Food,Quantity,Units,Price,FDC,link to price
0,McDonald's,Big Mac,7.60,ounces,6.21,170720,McDonald's App
1,,McChicken,131.00,grams,3.21,173305,McDonald's App
2,,10 pc McNuggets,159.00,grams,5.74,173297,McDonald's App
3,,Double Quarter Pounder w Cheese,280.00,grams,8.04,172066,McDonald's App
4,,Medium Fries,117.00,grams,3.67,170721,McDonald's App
5,,Filet o Fish,134.00,grams,6.20,170319,McDonald's App
6,,Cheeseburger,119.00,grams,2.03,170320,McDonald's App
7,,McChicken,95.00,grams,2.03,170717,McDonald's App
8,,Egg McMuffin,137,grams,2.79,173307,McDonald's App
9,,Sausage McMuffin with Egg,165.00,grams,2.99,172068,McDonald's App


In [6]:
import fooddatacentral as fdc

# Query to fdc and get dataframe containing nutritional data of foods using their fdc_ids
food_to_nutrients = {}
for food in df['Food'].tolist():
    try:
        fdc_id = int((df[df['Food'] == food].FDC).iloc[0])
        food_to_nutrients[food] = fdc.nutrients(apikey, fdc_id).Quantity
    except AttributeError: 
        warnings.warn("Couldn't find FDC Code %s for food %s." % (food,FDC))

D = pd.DataFrame(food_to_nutrients,dtype=float)
D

,Big Mac,McChicken,10 pc McNuggets,Double Quarter Pounder w Cheese,Medium Fries,Filet o Fish,Cheeseburger,Egg McMuffin,Sausage McMuffin with Egg,Side Salad
10:0,NaN,0.006,0.006,NaN,0.011,0.045,NaN,0.107,NaN,NaN
12:0,NaN,0.004,0.005,NaN,0.005,0.048,NaN,0.098,NaN,NaN
14:0,NaN,0.024,0.035,NaN,0.012,0.164,NaN,0.329,NaN,NaN
14:1,NaN,0.006,0.010,NaN,0.001,0.015,NaN,0.033,NaN,NaN
15:0,NaN,0.004,0.006,NaN,0.005,0.019,NaN,0.037,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
Vitamin K (Dihydrophylloquinone),NaN,NaN,NaN,NaN,48.600,0.000,NaN,NaN,NaN,NaN
Vitamin K (phylloquinone),NaN,NaN,NaN,NaN,16.000,4.900,NaN,NaN,NaN,NaN
Vitamins and Other Components,0.00,0.000,0.000,0.00,0.000,0.000,0.00,0.000,0.00,0.00
Water,51.30,46.110,47.130,50.34,36.630,45.760,45.00,52.580,49.96,93.93


### 1.4 #A Nutritional content of different foods

For all the foods you're considering you'll need to be able to describe their nutritional content, in terms that allow you to compare with recommended daily allowances. We'll provide points to US government recommendations, but many other countries have their own recommendations, which may be better suited to their populations.

In [7]:
bmin = pd.read_csv('./diet_minimums.csv').set_index('Nutrition').iloc[:,2:]
bmax = pd.read_csv('./diet_maximums.csv').set_index('Nutrition').iloc[:,2:]

In [8]:
import fooddatacentral as fdc

# Convert data type to float
df['Quantity'] = pd.to_numeric(df['Quantity'], downcast="float") 
df['Price'] = pd.to_numeric(df['Price'], downcast="float")

# Convert food quantities to FDC units
df['FDC Quantity'] = df[['Quantity', 'Units']].T.apply(lambda x: fdc.units(x['Quantity'], x['Units']))
df['FDC Price'] = df['Price']/df['FDC Quantity']
df.dropna()

# Get series of food with their price/hectogram
p = df.groupby('Food')['FDC Price'].min()

p

Food
10 pc McNuggets                    3.6100627491309205 / hectogram
Big Mac                             2.882254050016064 / hectogram
Cheeseburger                       1.7058823288989668 / hectogram
Double Quarter Pounder w Cheese    2.8714285578046526 / hectogram
Egg McMuffin                       2.0364963225204575 / hectogram
Filet o Fish                         4.62686552930234 / hectogram
McChicken                           2.136842075147127 / hectogram
Medium Fries                        3.136752201960637 / hectogram
Sausage McMuffin with Egg          1.8121212179010564 / hectogram
Side Salad                         2.0574712205207213 / hectogram
Name: FDC Price, dtype: object

In [9]:
# Remove price/hectogram and drop NAN values
p = p.apply(lambda x:x.magnitude).dropna()

# Compile list that we have both prices and nutritional info for; drop if either missing
selection = list(set(p.index.tolist()).intersection(D.columns.tolist()))
p = p[selection]

p

Food
Double Quarter Pounder w Cheese    2.871429
Big Mac                            2.882254
Medium Fries                       3.136752
Sausage McMuffin with Egg          1.812121
Side Salad                         2.057471
Egg McMuffin                       2.036496
Filet o Fish                       4.626866
McChicken                          2.136842
10 pc McNuggets                    3.610063
Cheeseburger                       1.705882
Name: FDC Price, dtype: float64

In [10]:
# Rename folate row
D.loc['Folate, DFE'] = D.loc['Folate, total']

# convert "Vitamin A, IU" to "Vitamin A, RAE"
if D.loc['Vitamin A, IU'].any():
    D.loc['Vitamin A, RAE'] = D.loc['Vitamin A, IU'] * 0.3

In [11]:
# Drop nutritional information for foods we don't know the price of,
# and replace missing nutrients NAN with zeros.
Aall = D[p.index].fillna(0)

# Drop rows of A that we don't have constraints for.
Amin = Aall.loc[bmin.index]
Amax = Aall.loc[bmax.index]

# Not Limiting sodium values for mcdonalds
A = pd.concat([Amin, Amax])
A

,Double Quarter Pounder w Cheese,Big Mac,Medium Fries,Sausage McMuffin with Egg,Side Salad,Egg McMuffin,Filet o Fish,McChicken,10 pc McNuggets,Cheeseburger
Nutrition,,,,,,,,,,
Energy,1096.000,1076.000,1352.000,1146.000,85.000,955.000,1182.000,1140.000,1263.000,1100.000
Protein,16.960,11.820,3.410,12.590,1.030,13.640,11.260,10.430,15.790,12.970
"Fiber, total dietary",1.000,1.600,3.900,0.900,1.600,1.100,1.400,1.300,0.000,1.100
"Folate, DFE",45.000,46.000,0.000,0.000,57.000,79.000,21.000,0.000,0.000,59.000
"Calcium, Ca",106.000,116.000,19.000,172.000,24.000,192.000,120.000,88.000,11.000,167.000
"Carbohydrate, by difference",14.430,20.080,42.580,17.280,4.300,21.670,26.390,27.970,15.090,27.810
"Iron, Fe",2.180,2.000,0.800,1.970,0.650,2.300,1.560,1.740,0.890,2.350
"Magnesium, Mg",21.000,20.000,37.000,18.000,0.000,20.000,27.000,20.000,24.000,20.000
Niacin,4.213,3.384,3.220,2.930,0.208,3.102,2.220,3.942,7.400,4.026


In [12]:
A.loc['Folate, DFE']

Double Quarter Pounder w Cheese    45.0
Big Mac                            46.0
Medium Fries                        0.0
Sausage McMuffin with Egg           0.0
Side Salad                         57.0
Egg McMuffin                       79.0
Filet o Fish                       21.0
McChicken                           0.0
10 pc McNuggets                     0.0
Cheeseburger                       59.0
Name: Folate, DFE, dtype: float64

In [13]:
A.loc['Vitamin A, RAE']

Double Quarter Pounder w Cheese     60.0
Big Mac                             56.4
Medium Fries                         0.0
Sausage McMuffin with Egg          100.5
Side Salad                         855.0
Egg McMuffin                         0.0
Filet o Fish                         0.0
McChicken                            0.0
10 pc McNuggets                      0.0
Cheeseburger                        72.9
Name: Vitamin A, RAE, dtype: float64

In [14]:
b = pd.concat([bmin, -bmax])
b

,C 1-3,F 4-8,M 4-8,F 9-13,M 9-13,F 14-18,M 14-18,F 19-30,M 19-30,F 31-50,M 31-50,F 51+,M 51+
Nutrition,,,,,,,,,,,,,
Energy,1000.0,1200.0,1400.0,1600.0,1800.0,1800.0,2200.0,2000.0,2400.0,1800.0,2200.0,1600.0,2000.0
Protein,13.0,19.0,19.0,34.0,34.0,46.0,52.0,46.0,56.0,46.0,56.0,46.0,56.0
"Fiber, total dietary",14.0,16.8,19.6,22.4,25.2,25.2,30.8,28.0,33.6,25.2,30.8,22.4,28.0
"Folate, DFE",150.0,200.0,200.0,300.0,300.0,400.0,400.0,400.0,400.0,400.0,400.0,400.0,400.0
"Calcium, Ca",700.0,1000.0,1000.0,1300.0,1300.0,1300.0,1300.0,1000.0,1000.0,1000.0,1000.0,1200.0,1000.0
"Carbohydrate, by difference",130.0,130.0,130.0,130.0,130.0,130.0,130.0,130.0,130.0,130.0,130.0,130.0,130.0
"Iron, Fe",7.0,10.0,10.0,8.0,8.0,15.0,11.0,18.0,8.0,18.0,8.0,8.0,8.0
"Magnesium, Mg",80.0,130.0,130.0,240.0,240.0,360.0,410.0,310.0,400.0,320.0,420.0,320.0,420.0
Niacin,6.0,8.0,8.0,12.0,12.0,14.0,16.0,14.0,16.0,14.0,16.0,14.0,16.0


### 1.5 #A Solution Diet

In [15]:
from  scipy.optimize import linprog

sex_age_group = 'M 19-30'

# negate A and b to convert (Ax <= b) to (Ax >= b)
result = linprog(p, -A, -b[sex_age_group], method='interior-point')
result

     con: array([], dtype=float64)
     fun: 43.840720391002606
 message: 'Optimization terminated successfully.'
     nit: 15
   slack: array([1.61382891e+04, 3.90214370e+01, 1.39335695e+01, 1.35849518e-08,
       1.68567125e+01, 4.24129392e+02, 1.16585092e+01, 6.47946603e+01,
       3.34336414e+01, 1.30833212e+03, 2.61519023e+03, 4.72254556e-01,
       1.76436011e+00, 9.04532307e+02, 1.13860921e-10, 2.80200433e+00,
       1.45054457e-09, 7.10560499e-11, 4.27767983e+01, 1.61882176e+00,
       6.99670540e+03])
  status: 0
 success: True
       x: array([4.39981708e-10, 1.05386457e-10, 1.01735499e+01, 1.34221683e-08,
       1.87262041e+00, 1.62796804e+00, 1.60349487e-10, 7.23050778e-11,
       4.44161245e-11, 2.79069766e+00])

What is the minimum cost diet for the population you're concerned with? How much does it cost, and of what does it consist? How does it vary with age, sex, and level of activity?

In [16]:
print("Cost of %s diet for %s is $%4.2f per day." % (diet_type, sex_age_group, result.fun))

Cost of mcdonalds diet for M 19-30 is $43.84 per day.


In [17]:
diet = pd.Series(result.x,index=p.index)
outcome_df = pd.DataFrame({"Outcome":np.abs(A).dot(diet),"Recommendation for %s" % (sex_age_group):np.abs(b[sex_age_group])})
excess = outcome_df.diff(axis=1).iloc[:,1]
print("Diet in hectograms")
print(diet[diet >= 1e-6]) #smaller than this are treated as zero
print("\nCost of a %s diet for %s is $%4.2f per day." % (diet_type, sex_age_group, result.fun))
print("\nConstraining nutrients are:")
print(excess.loc[np.abs(excess) < 1e-6].index.tolist())
outcome_df

Diet in hectograms
Food
Medium Fries    10.173550
Side Salad       1.872620
Egg McMuffin     1.627968
Cheeseburger     2.790698
dtype: float64

Cost of a mcdonalds diet for M 19-30 is $43.84 per day.

Constraining nutrients are:
['Folate, DFE', 'Vitamin B-12', 'Vitamin C, total ascorbic acid', 'Vitamin E (alpha-tocopherol)']


,Outcome,Recommendation for M 19-30
Nutrition,,
Energy,18538.289109,2400.0
Protein,95.021437,56.0
"Fiber, total dietary",47.533570,33.6
"Folate, DFE",400.000000,400.0
"Calcium, Ca",1016.856713,1000.0
"Carbohydrate, by difference",554.129392,130.0
"Iron, Fe",19.658509,8.0
"Magnesium, Mg",464.794660,400.0
Niacin,49.433641,16.0


In [18]:
for nutrition, value in A.sum(axis=1).items():
    if value == 0:
        print(f"Fail to find solution.\n{diet_type} diet does not provide any {nutrition}!")

## Minimum Cost Diet Function

In [19]:
# API
user = "project2-casimir-funk"
apikey = "V0mOAdVrSineT4d2VTMTJUNPAsBTnjzUAU6e6H6V" # inIyO1begWSRqsYtxS7m6p09PSyq7Qiw7fxzV2qN"

# File with private keys for relevant service account to authenticate
# and access google spreadsheets
!gpg --passphrase "casimir" -d project2-casimir-funk-306520-ae3703c35882.json.gpg > project2-casimir-funk-306520-ae3703c35882.json

serviceacct = {'project2-casimir-funk':'project2-casimir-funk-306520-ae3703c35882.json'}
sheet_url = "https://docs.google.com/spreadsheets/d/1XqnsamAhXq2xnktB_rJ8h8ovvpFQdWWcDkx_1Yrfqdg/edit#gid=0"

gpg: AES256 encrypted data
gpg: encrypted with 1 passphrase


In [20]:
import pandas as pd
import numpy as np
from eep153_tools import read_sheets
import fooddatacentral as fdc
from  scipy.optimize import linprog


def minimum_cost_diet(diet_type, sex_age_group):
    bmin = pd.read_csv('./diet_minimums.csv').set_index('Nutrition').iloc[:,2:]
    bmax = pd.read_csv('./diet_maximums.csv').set_index('Nutrition').iloc[:,2:]
    
    # Accessing the chosen diet from google sheets and convert to pandas.dataframe
    data = read_sheets(key=sheet_url, json_creds=serviceacct[user], sheet=diet_type)
    headers = data.iloc[0]
    df  = pd.DataFrame(data.values[1:], columns=headers)

    # Convert data type to float
    df['Quantity'] = pd.to_numeric(df['Quantity'], downcast="float") 
    df['Price'] = pd.to_numeric(df['Price'], downcast="float")

    # Convert food quantities to FDC units
    df['FDC Quantity'] = df[['Quantity', 'Units']].T.apply(lambda x: fdc.units(x['Quantity'], x['Units']))
    df['FDC Price'] = df['Price']/df['FDC Quantity']
    df.dropna()

    # Prices of food
    p = df.groupby('Food')['FDC Price'].min()
    p = p.apply(lambda x:x.magnitude).dropna()

    # Query to fdc and get dataframe containing nutritional data of foods using their fdc_ids
    food_to_nutrients = {}
    for food in df['Food'].tolist():
        try:
            fdc_id = int((df[df['Food'] == food].FDC).iloc[0])
            food_to_nutrients[food] = fdc.nutrients(apikey, fdc_id).Quantity
        except AttributeError: 
            warnings.warn("Couldn't find FDC Code %s for food %s." % (food,FDC))
    nutritional_df = pd.DataFrame(food_to_nutrients,dtype=float)
    
    # rename "Folate, total" to "Folate, DFE"
    if nutritional_df.loc['Folate, total'].any():
        nutritional_df.loc['Folate, DFE'] = nutritional_df.loc['Folate, total']
    
    # convert "Vitamin A, IU" to "Vitamin A, RAE"
    if nutritional_df.loc['Vitamin A, IU'].any():
        nutritional_df.loc['Vitamin A, RAE'] = nutritional_df.loc['Vitamin A, IU'] * 0.3
    
    # Drop nutritional information for foods we don't know the price of, and replace missing nutrients with zeros.
    Aall = nutritional_df[p.index].fillna(0)

    # Drop rows of A that we don't have constraints for.
    Amin = Aall.loc[bmin.index]
    Amax = Aall.loc[bmax.index]

    # Negate Sodium values to fit constraint
    A = pd.concat([Amin,-Amax])

    # Solving the diet problem
    b = pd.concat([bmin, -bmax])
    result = linprog(p, -A, -b[sex_age_group], method='interior-point')
    
    if result.success:
        tolerance = 0.05
        diet = pd.Series(result.x,index=p.index)
        outcome_df = pd.DataFrame({"Outcome":np.abs(A).dot(diet),"Recommendation for %s" % (sex_age_group):np.abs(b[sex_age_group])})
        excess = outcome_df.diff(axis=1).iloc[:,1]
        print("Diet in hectograms")
        print(diet[diet >= tolerance]) #smaller than this are treated as zero
        print("\nCost of a %s diet for %s is $%4.2f per day." % (diet_type, sex_age_group, result.fun))
        print("\nConstraining nutrients are:")
        print(excess.loc[np.abs(excess) < tolerance].index.tolist())
        return outcome_df
        
    else:
        # print missing nutrients from input diet type
        for nutrition, value in A.sum(axis=1).items():
            if value == 0:
                print(f"Fail to find solution.\n{diet_type} diet does not provide any {nutrition}!")
        print(f"Fail to find solution with result status {result.status}")

### Calling the function

More info on scipy linprog: https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.linprog.html

In [21]:
minimum_cost_diet(diet_type='omnivore', sex_age_group='M 19-30')

Diet in hectograms
Food
Milk            3.234613
Spinach         7.389163
Wheat Cereal    6.512360
dtype: float64

Cost of a omnivore diet for M 19-30 is $6.98 per day.

Constraining nutrients are:
['Protein', 'Calcium, Ca', 'Vitamin E (alpha-tocopherol)']


,Outcome,Recommendation for M 19-30
Nutrition,,
Energy,3326.868096,2400.0
Protein,56.000001,56.0
"Fiber, total dietary",40.351891,33.6
"Folate, DFE",1433.497537,400.0
"Calcium, Ca",1000.000001,1000.0
"Carbohydrate, by difference",621.879428,130.0
"Iron, Fe",63.462074,8.0
"Magnesium, Mg",967.973096,400.0
Niacin,125.952153,16.0


In [22]:
minimum_cost_diet(diet_type='omnivore', sex_age_group='F 19-30')

Diet in hectograms
Food
Milk            3.234613
Spinach         7.389163
Wheat Cereal    3.809657
dtype: float64

Cost of a omnivore diet for F 19-30 is $6.93 per day.

Constraining nutrients are:
['Protein', 'Calcium, Ca', 'Vitamin E (alpha-tocopherol)']


,Outcome,Recommendation for F 19-30
Nutrition,,
Energy,2326.868063,2000.0
Protein,46.000000,46.0
"Fiber, total dietary",30.351890,28.0
"Folate, DFE",1433.497537,400.0
"Calcium, Ca",1000.000000,1000.0
"Carbohydrate, by difference",381.636180,130.0
"Iron, Fe",45.435046,18.0
"Magnesium, Mg",808.513634,310.0
Niacin,75.900801,14.0


In [23]:
minimum_cost_diet(diet_type='pescatarian', sex_age_group='M 19-30')

Diet in hectograms
Food
Milk            3.234613
Spinach         7.389162
Wheat Cereal    6.512359
dtype: float64

Cost of a pescatarian diet for M 19-30 is $6.98 per day.

Constraining nutrients are:
['Protein', 'Calcium, Ca', 'Vitamin E (alpha-tocopherol)']


,Outcome,Recommendation for M 19-30
Nutrition,,
Energy,3326.868126,2400.0
Protein,56.000006,56.0
"Fiber, total dietary",40.351893,33.6
"Folate, DFE",1433.497537,400.0
"Calcium, Ca",1000.000047,1000.0
"Carbohydrate, by difference",621.879399,130.0
"Iron, Fe",63.462070,8.0
"Magnesium, Mg",967.973056,400.0
Niacin,125.952139,16.0


In [24]:
minimum_cost_diet(diet_type='pescatarian', sex_age_group='F 19-30')

Diet in hectograms
Food
Milk            3.234608
Spinach         7.389162
Wheat Cereal    3.809648
dtype: float64

Cost of a pescatarian diet for F 19-30 is $6.93 per day.

Constraining nutrients are:
['Protein', 'Calcium, Ca', 'Vitamin E (alpha-tocopherol)']


,Outcome,Recommendation for F 19-30
Nutrition,,
Energy,2326.866600,2000.0
Protein,46.000004,46.0
"Fiber, total dietary",30.351884,28.0
"Folate, DFE",1433.497594,400.0
"Calcium, Ca",1000.000096,1000.0
"Carbohydrate, by difference",381.635535,130.0
"Iron, Fe",45.434990,18.0
"Magnesium, Mg",808.513094,310.0
Niacin,75.900623,14.0


In [25]:
minimum_cost_diet(diet_type='vegetarian', sex_age_group='M 19-30')

Diet in hectograms
Food
Milk            3.234613
Spinach         7.389163
Wheat Cereal    6.512360
dtype: float64

Cost of a vegetarian diet for M 19-30 is $6.98 per day.

Constraining nutrients are:
['Protein', 'Calcium, Ca', 'Vitamin E (alpha-tocopherol)']


,Outcome,Recommendation for M 19-30
Nutrition,,
Energy,3326.868063,2400.0
Protein,56.000000,56.0
"Fiber, total dietary",40.351890,33.6
"Folate, DFE",1433.497537,400.0
"Calcium, Ca",1000.000000,1000.0
"Carbohydrate, by difference",621.879423,130.0
"Iron, Fe",63.462073,8.0
"Magnesium, Mg",967.973093,400.0
Niacin,125.952152,16.0


In [26]:
minimum_cost_diet(diet_type='vegetarian', sex_age_group='F 19-30')

Diet in hectograms
Food
Milk            3.234609
Spinach         7.389162
Wheat Cereal    3.809656
dtype: float64

Cost of a vegetarian diet for F 19-30 is $6.93 per day.

Constraining nutrients are:
['Protein', 'Calcium, Ca', 'Vitamin E (alpha-tocopherol)']


,Outcome,Recommendation for F 19-30
Nutrition,,
Energy,2326.868901,2000.0
Protein,46.000014,46.0
"Fiber, total dietary",30.351911,28.0
"Folate, DFE",1433.497728,400.0
"Calcium, Ca",1000.000090,1000.0
"Carbohydrate, by difference",381.636162,130.0
"Iron, Fe",45.435041,18.0
"Magnesium, Mg",808.513590,310.0
Niacin,75.900767,14.0


In [27]:
minimum_cost_diet(diet_type='vegan', sex_age_group='M 19-30')

Diet in hectograms
Food
Bananas         1.719923
Canned Beans    0.848927
Spinach         7.389162
Wheat Cereal    2.009175
dtype: float64

Cost of a vegan diet for M 19-30 is $7.12 per day.

Constraining nutrients are:
['Energy', 'Protein', 'Calcium, Ca', 'Vitamin E (alpha-tocopherol)']


,Outcome,Recommendation for M 19-30
Nutrition,,
Energy,2400.000448,2400.0
Protein,56.000005,56.0
"Fiber, total dietary",39.022745,33.6
"Folate, DFE",1453.871852,400.0
"Calcium, Ca",1000.000007,1000.0
"Carbohydrate, by difference",292.446870,130.0
"Iron, Fe",36.481219,8.0
"Magnesium, Mg",729.450851,400.0
Niacin,42.788573,16.0


In [28]:
minimum_cost_diet(diet_type='vegan', sex_age_group='F 19-30')

Diet in hectograms
Food
Bananas          1.073405
Canned Beans     0.746087
Green Cabbage    2.670783
Spinach          7.191814
Wheat Cereal     0.913541
dtype: float64

Cost of a vegan diet for F 19-30 is $7.08 per day.

Constraining nutrients are:
['Energy', 'Protein', 'Calcium, Ca', 'Phosphorus, P', 'Vitamin E (alpha-tocopherol)']


,Outcome,Recommendation for F 19-30
Nutrition,,
Energy,2000.000458,2000.0
Protein,46.000000,46.0
"Fiber, total dietary",36.637641,28.0
"Folate, DFE",1527.961704,400.0
"Calcium, Ca",1000.000001,1000.0
"Carbohydrate, by difference",181.489062,130.0
"Iron, Fe",29.032912,18.0
"Magnesium, Mg",677.976410,310.0
Niacin,22.952636,14.0


In [29]:
minimum_cost_diet(diet_type='gluten-free', sex_age_group='M 19-30')

Diet in hectograms
Food
Canned Beans     1.004978
Green Cabbage    5.251043
Milk             0.290157
Pork Liver       0.994125
Spinach          7.001154
Sugar            0.500179
dtype: float64

Cost of a gluten-free diet for M 19-30 is $7.35 per day.

Constraining nutrients are:
['Protein', 'Calcium, Ca', 'Carbohydrate, by difference', 'Thiamin', 'Vitamin E (alpha-tocopherol)', 'Zinc, Zn']


,Outcome,Recommendation for M 19-30
Nutrition,,
Energy,3086.818763,2400.0
Protein,56.000000,56.0
"Fiber, total dietary",34.057529,33.6
"Folate, DFE",1818.892862,400.0
"Calcium, Ca",1000.000003,1000.0
"Carbohydrate, by difference",130.000001,130.0
"Iron, Fe",45.970871,8.0
"Magnesium, Mg",666.157279,400.0
Niacin,21.782046,16.0


In [30]:
minimum_cost_diet(diet_type='gluten-free', sex_age_group='F 19-30')

Diet in hectograms
Food
Canned Beans     1.142297
Green Cabbage    5.861300
Pork Liver       0.496426
Spinach          6.956062
Sugar            0.465376
dtype: float64

Cost of a gluten-free diet for F 19-30 is $7.24 per day.

Constraining nutrients are:
['Protein', 'Calcium, Ca', 'Carbohydrate, by difference', 'Thiamin', 'Vitamin E (alpha-tocopherol)']


,Outcome,Recommendation for F 19-30
Nutrition,,
Energy,2856.703953,2000.0
Protein,46.000001,46.0
"Fiber, total dietary",36.239219,28.0
"Folate, DFE",1734.169213,400.0
"Calcium, Ca",1000.000020,1000.0
"Carbohydrate, by difference",130.000002,130.0
"Iron, Fe",34.719633,18.0
"Magnesium, Mg",665.353621,310.0
Niacin,14.314246,14.0


In [31]:
minimum_cost_diet(diet_type='chipotle', sex_age_group='M 19-30')

Fail to find solution.
chipotle diet does not provide any Vitamin B-12!
Fail to find solution with result status 2


In [32]:
minimum_cost_diet(diet_type='mcdonalds', sex_age_group='M 19-30')

Fail to find solution with result status 2


### 1.6 #B Is your solution edible?

Turn your minimum cost diet into a meal. Share the recipe on piazza. If making your diet palatable requires adding additional ingredients (e.g. salt or other spices) besure to document this and account for it in the overall cost. (You may want to take pictures or video of preparation and eating). Whose meal is least costly? Whose is most tasty?

### 1.7 #B Meal Reviews

Get the recipe for the meal from some other group(s), and prepare the meal according to their recipe. Provide a review (follow-up to the piazza post with the recipe). How much did the ingredients cost? Would you recommend it to others? Eat it again? Eat it exclusively?

### 1.8 #C Sensitivity of Solution

In general, your solution will depend on relative prices of different kinds of foods. And these prices can be quite volatile; for example, the prices of fresh fruits and vegetables usually exhibit a great deal of seasonal variation. If the price of a particula rkind of food increases by x%, how much does price of diet change, and how does the composition of the diet change? Try and find an nice way to visualize these changes.

### 1.9 #B What is the total cost for population of interest?

You've been interested in a particular population. It is likely to consist of people of different sexes, ages, and levels of activity. How much food would be required to feed the entire population of interest? At what cost? (You may want to go back to the Population Project and look at counts of people by age and sex for different countries).

### 1.10 #B Review of Your Code

### 1.11 #B Review of Another Group's Code

### 1.12 #A Presentation